In [118]:
#Importing libraries
import numpy as np
import pandas as pd
import plotly.express as px
import requests

In [119]:
#Accessing the TfL API with list of all lines of a given mode of transport, tube as the selected mode as specified in the url
url_tube_lines = "https://api.tfl.gov.uk/Line/Mode/tube"
response_tl = requests.get(url_tube_lines)
data_tl = response_tl.json()

#Creating data frame of the information
df_tl = pd.DataFrame(data_tl)

#creating a list of the tube lines from the dataframe by iterating through the id column
TubeLines = []
for i in df_tl.loc[:, "id"]:
    TubeLines.append(i)

#creating a list of colours in the same order as the TubeLines list
tfl_colours = ['rgb(137, 78, 36)', 'rgb(220, 36, 31)', 'rgb(255, 206, 0)', 'rgb(0, 114, 41)', 'rgb(215, 153, 175)', 'rgb(134, 143, 152)', 'rgb(117, 16, 86)', 'rgb(0, 0, 0)', 
'rgb(0, 25, 168)', 'rgb(0, 160, 226)', 'rgb(118, 208, 189)']

#Joining the two lists together to make a dictionary using zip, the colours will be used to plot the map
colours_dict = {line:colour for (line,colour) in zip(TubeLines, tfl_colours)}



In [120]:
#creating the function that will be used with the TubeLines list to print out data specific to that tube line

def line_analysis(line):
    #Accessing the TfL API showing tube stations on the specified line
    url_stops = f"https://api.tfl.gov.uk/Line/{line}/StopPoints"
    response_stops = requests.get(url_stops)
    data_stops = response_stops.json()

    #Creating data frame from the information from the API about the tube stations
    df_stops = pd.DataFrame(data_stops)

    #Creating a new data frame using only the columns I need for analysis
    df_stops_merge = df_stops[["naptanId", "commonName", "lat", "lon"]]

    #Creating a data frame from the stations.csv - file from TfL containing information on stations
    df_stations = pd.read_csv("Stations.csv")  
    
    #Creating a new data frame using only the columns I need for analysis
    df_stations_cond = df_stations[["UniqueId", "FareZones", "Wifi", "BlueBadgeCarParking", "BlueBadgeCarParkSpaces", "TaxiRanksOutsideStation"]]
    
    #Renaming the UniqueId column to naptanId to match the tube stops data frame for a merge
    df_stations_merge = df_stations_cond.rename(columns={'UniqueId': 'naptanId'})
    
    #Adding the columns from stations.csv to the tube stops data frame based on the naptanId
    df_merge_1 = pd.merge(df_stops_merge, df_stations_merge, on='naptanId', how='left')
    
    #reading the toilet data csv file I created in TfLCSVs.ipynb using information from TfL's toilet csv
    df_Toilets = pd.read_csv("CustomTfLToilet.csv")
    
    #merging the toilet data csv with the previous data frame
    df_tube_line_data = pd.merge(df_merge_1, df_Toilets, on='naptanId', how='left')
    
    #replacing values in specified columns to show up correctly in hover template on plotted map
    df_tube_line_data["FareZones"] = df_tube_line_data["FareZones"].replace({np.nan: "n/a"})
    df_tube_line_data['Wifi'] = df_tube_line_data['Wifi'].replace([True, False, np.nan], ["Yes", "No",  "n/a"])
    df_tube_line_data["BlueBadgeCarParking"] = df_tube_line_data["BlueBadgeCarParking"].replace({True: "Yes", False: "None", np.nan: "n/a"})
    df_tube_line_data["BlueBadgeCarParkSpaces"] = df_tube_line_data["BlueBadgeCarParkSpaces"].replace({False: "None", np.nan: "n/a"})
    df_tube_line_data["TaxiRanksOutsideStation"] = df_tube_line_data["TaxiRanksOutsideStation"].replace({True: "Yes", False: "None", np.nan: "n/a"})
    df_tube_line_data["NumToilets"] = df_tube_line_data["NumToilets"].replace({True: "Yes", False: "None", np.nan: "n/a"})
    df_tube_line_data["HasAccessibleToilets"] = df_tube_line_data["HasAccessibleToilets"].replace({True: "Yes", False: "None", np.nan: "n/a"})
    
    
    #plotting a scatter map using the data from the data frame, including a custom data list which will be shown on hover, as specified in the hover template
    fig_tube_stops = px.scatter_map(df_tube_line_data, lat="lat", lon="lon", hover_name="commonName", zoom=9.5, 
                         custom_data=['commonName', 'Wifi', 'NumToilets', 'HasAccessibleToilets', 'FareZones', 'BlueBadgeCarParking', 'BlueBadgeCarParkSpaces', 'TaxiRanksOutsideStation'])      
    
    #cutomising the appearance of the scatter map, adding a title and styling including tube line colour from the dictionary created earlier
    fig_tube_stops.update_layout(title= f'{line.capitalize()} Line Stations', hovermode='closest', map_style="light", hoverlabel_bgcolor='rgb(0, 25, 168)')    
    fig_tube_stops.update_traces(marker=dict(color= colours_dict[f"{line}"], size=8.5))
    
    #creating a template for the text displayed when hovering over the station markers using the custom data
    fig_tube_stops.update_traces(hovertemplate = 
                      "<b>%{customdata[0]}</b><br>" + 
                      "<br><b>Wifi:</b> %{customdata[1]}" + 
                      "<br><b>Toilets:</b> %{customdata[2]}" +
                      "<br><b>Accessible toilets:</b> %{customdata[3]}" +
                      "<br><b>Zone:</b> %{customdata[4]}" +
                      "<br><b>Accessible parking:</b> %{customdata[5]}" +
                      "<br><b>Accessible parking spaces:</b> %{customdata[6]}" +
                      "<br><b>Accessible Taxi rank:</b> %{customdata[7]}")
                      
    #displaying the scatter map
    fig_tube_stops.show()

    #Using for loops to count how many stations have different amenities and get zone information
    wifi_count = sum(x == "Yes" for x in df_tube_line_data["Wifi"])
    bb_parking_count = sum(x == "Yes" for x in df_tube_line_data["BlueBadgeCarParking"])
    toilet_count = sum(x != "n/a" for x in df_tube_line_data["NumToilets"])
    acc_toilet_count = sum(x == "Yes" for x in df_tube_line_data["HasAccessibleToilets"])
    taxi_rank_count = sum(x == "Yes" for x in df_tube_line_data["TaxiRanksOutsideStation"])
    stop_zones = []
    [stop_zones.append(x) for x in df_tube_line_data["FareZones"] if x not in stop_zones and x != "n/a"]


    #printing information about the tube line using information from the data frame and the above for loops, with conditional statements for different numbers
    print("From the available data, it has been found that:")
    print(f"The {line.capitalize()} Line has {len(df_tube_line_data)} stops.")
    if stop_zones:
        print(f"The {line.capitalize()} Line goes through zones {', '.join(stop_zones[0:-1])} and {stop_zones[-1]}." if len(stop_zones) != 1 else f"The {line.capitalize()} Line goes through zone {stop_zones[0]}.")
    else:
        print("There is no zone data available.")
    print(f"{wifi_count} stations have Wifi." if wifi_count != 1 else f"{wifi_count} station has Wifi.")
    print(f"{toilet_count} stations have toilets." if toilet_count != 1 else f"{toilet_count} station has toilets.")
    print(f"{acc_toilet_count} stations have accessible toilets." if acc_toilet_count != 1 else f"{acc_toilet_count} station has accessible toilets.")
    print(f"{bb_parking_count} stations have blue badge parking." if bb_parking_count != 1 else f"{bb_parking_count} station has accessible toilets.")
    print(f"{taxi_rank_count} stations have taxi ranks." if taxi_rank_count != 1 else f"{taxi_rank_count} station has taxi ranks.")



In [121]:
#printing basic information found about the tube lines from the tube lines API using string formatting
print(f"There are {len(TubeLines)} tube lines in the TfL underground network.")
print(f"These lines are: {', '.join([x.title() for x in TubeLines[0:-1]])} and {TubeLines[-1].title()}.")

for x in TubeLines:
    line_analysis(x)

There are 11 tube lines in the TfL underground network.
These lines are: Bakerloo, Central, Circle, District, Hammersmith-City, Jubilee, Metropolitan, Northern, Piccadilly, Victoria and Waterloo-City.


From the available data, it has been found that:
The Bakerloo Line has 25 stops.
The Bakerloo Line goes through zones 1 and 2.
10 stations have Wifi.
2 stations have toilets.
0 stations have accessible toilets.
0 stations have blue badge parking.
0 stations have taxi ranks.


From the available data, it has been found that:
The Central Line has 49 stops.
The Central Line goes through zones 4, 5, 2, 1, 6, 3, 3|4, 2|3 and 1|2.
37 stations have Wifi.
23 stations have toilets.
3 stations have accessible toilets.
9 stations have blue badge parking.
3 stations have taxi ranks.


From the available data, it has been found that:
The Circle Line has 36 stops.
The Circle Line goes through zones 1, 2 and 1|2.
25 stations have Wifi.
2 stations have toilets.
1 station has accessible toilets.
0 stations have blue badge parking.
0 stations have taxi ranks.


From the available data, it has been found that:
The District Line has 60 stops.
The District Line goes through zones 3, 1, 2|3, 5, 2, 1|2, 3|4, 6 and 4.
41 stations have Wifi.
11 stations have toilets.
3 stations have accessible toilets.
0 stations have blue badge parking.
2 stations have taxi ranks.


...

From the available data, it has been found that:
The Metropolitan Line has 34 stops.
The Metropolitan Line goes through zones 1, 9, 7, 5, 2, 6, 6|7 and 4.
22 stations have Wifi.
18 stations have toilets.
6 stations have accessible toilets.
5 stations have blue badge parking.
0 stations have taxi ranks.


From the available data, it has been found that:
The Northern Line has 52 stops.
The Northern Line goes through zones 1, 2|3, 4, 3, 2, 5, 3|4 and 1|2.
40 stations have Wifi.
8 stations have toilets.
5 stations have accessible toilets.
3 stations have blue badge parking.
1 station has taxi ranks.


From the available data, it has been found that:
The Piccadilly Line has 53 stops.
The Piccadilly Line goes through zones 3, 4, 2, 3|4, 1, 5, 1|2, 6, 5|6 and 2|3.
46 stations have Wifi.
21 stations have toilets.
4 stations have accessible toilets.
7 stations have blue badge parking.
0 stations have taxi ranks.


From the available data, it has been found that:
The Victoria Line has 16 stops.
The Victoria Line goes through zones 1 and 2.
5 stations have Wifi.
1 station has toilets.
0 stations have accessible toilets.
0 stations have blue badge parking.
0 stations have taxi ranks.


From the available data, it has been found that:
The Waterloo-city Line has 2 stops.
There is no zone data available.
0 stations have Wifi.
0 stations have toilets.
0 stations have accessible toilets.
0 stations have blue badge parking.
0 stations have taxi ranks.
